In [6]:
import mplusdb
import time
import requests
import blizzard_api
import sys
import blizzard_credentials

In [7]:
auth = blizzard_credentials.Credentials('.api_tokens')
access_token = auth.access_token

In [3]:
mdb = mplusdb.MplusDatabase('.db_config')
realms = mdb.get_utility_table('realm')

In [4]:
realms

,cluster_id,realm_id,name,name_slug,region,locale,timezone
0,3694,1,Lightbringer,lightbringer,1,enUS,America/Los_Angeles
1,1168,2,Cenarius,cenarius,1,enUS,America/Los_Angeles
2,151,3,Uther,uther,1,enUS,America/Chicago
3,4,4,Kilrogg,kilrogg,1,enUS,America/Los_Angeles
4,5,5,Proudmoore,proudmoore,1,enUS,America/Los_Angeles
...,...,...,...,...,...,...,...
551,985,1049,Wrathbringer,wrathbringer,4,zhTW,Asia/Taipei
552,999,1054,Arthas,arthas,4,zhTW,Asia/Taipei
553,963,1056,Quel'dorei,queldorei,4,zhTW,Asia/Taipei
554,964,1057,Icecrown,icecrown,4,zhTW,Asia/Taipei


In [16]:
call_factory = blizzard_api.UrlFactory(
    region = 'us', 
    access_token = access_token)

realm_index_call = call_factory.get_connected_realm_index_call()
response = requests.get(realm_index_call)
realm_index_json = response.json()

parser = blizzard_api.ResponseParser()
cluster_ids = parser.parse_connected_realm_index_json(realm_index_json)

In [21]:
print(len(cluster_ids))

84


In [32]:
old_cluster_ids = realms.loc[realms.region==1, 'cluster_id']
old_cluster_ids = sorted(list(set(old_cluster_ids)))

cluster_ids = [int(i) for i in cluster_ids]

print("new clusters 9/2020:")
for cluster_id in cluster_ids:
    if cluster_id not in old_cluster_ids:
        print(cluster_id)
print('-'*10)
print("clusters merged into others 9/2020:")
for old_cluster_id in old_cluster_ids:
    if old_cluster_id not in cluster_ids:
        print(old_cluster_id)

new clusters 9/2020:
----------
clusters merged into others 9/2020:
7
10
51
70
74
85
87
90
91
98
101
105
116
119
123
128
153
156
159
1069
1146
1165
1169
1173
1174
1182
3677
3722
3724
3728
3729


In [9]:
help(blizzard_api)

Help on module blizzard_api:

NAME
    blizzard_api - Classes to query and parse World of Warcraft  (WoW) M+ game data.

DESCRIPTION
    Blizzard/WoW API docs:
    https://develop.battle.net/documentation/world-of-warcraft/game-data-apis

CLASSES
    builtins.object
        Caller
        KeyRun
        KeyRunLeaderboard
        RealmRecord
        ResponseParser
        RosterMember
        UrlFactory
    
    class Caller(builtins.object)
     |  Abstracts API interactions into a high-level interface.
     |  
     |  Methods defined here:
     |  
     |  __init__(self, access_token=None)
     |      Inits wtih access token. If no token, tries to get one.
     |  
     |  get_class_spec_table(self)
     |      Gets class/spec table.
     |  
     |  get_leaderboard(self, region, realm, dungeon, period)
     |      Gets leaderboard for specified region/realm/dungeon/period.
     |  
     |  get_spec_by_index(self, spec_index)
     |      Returns full info for spec.
     |  
     |  g